### **What is Chaining in LangChain?**

In **LangChain**, chaining refers to connecting multiple steps or components together to create a complete pipeline or workflow. Each step takes input, processes it, and passes the output to the next step. For example:

1. First, retrieve data from a database.
2. Pass the data to a language model to generate a response.
3. Finally, send the response to the user.

This allows developers to build **complex applications** by combining simple components like prompts, tools, and chains.

### **What is LCEL (LangChain Expression Language)?**

LCEL (LangChain Expression Language) is a **simple syntax** used in LangChain to write expressions that dynamically generate prompts or extract data. It’s designed to make it easier to manipulate and format text or data within chains.

For example:

```python
template = "Hello, {name}! Today is {day}."
filled_template = template.format(name="John", day="Monday")
```

LCEL helps with such dynamic behavior inside LangChain applications. It simplifies tasks like **customizing prompts** and **manipulating inputs/outputs**.


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20")
gemini_model.invoke("what is capital of USA?").content

'The capital of the USA is **Washington, D.C.**'

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

In [6]:
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate(
    template="can you give me a detail explaination of {topic}",
    input_variables=['topic']
)

## **Simple LLM chain**

In [7]:
chain=prompt | gemini_model | parser

In [8]:
chain.invoke({"topic": "machine learning"})

'Machine Learning (ML) is a subset of Artificial Intelligence (AI) that empowers computer systems to "learn" from data without being explicitly programmed. Instead of writing specific instructions for every possible scenario, you feed a machine learning algorithm a large amount of data, and it learns patterns, relationships, and rules from that data to perform a task or make predictions.\n\nLet\'s break down this detailed explanation:\n\n---\n\n### I. The Core Idea: Learning from Data\n\nImagine teaching a child to recognize a cat. You don\'t give them a list of rules like "if it has pointy ears AND whiskers AND meows, it\'s a cat." Instead, you show them many pictures of cats, and many pictures of non-cats, and say "This is a cat," or "This is not a cat." Over time, the child learns to identify a cat on their own, even if they see a breed they\'ve never encountered before.\n\nMachine learning works in a similar way:\n\n1.  **Traditional Programming:** You write explicit rules.\n    * 

In [9]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## **Sequential LLM chain**

In [12]:
text= """In November 2023, xAI began previewing Grok as a chatbot to selected people,[10] with participation in the early access program being limited to paid X Premium users.[11]

It was announced that once the bot was out of early beta, it would only be available to higher tier X Premium+ subscribers.[12]

At the time of the preview, xAI described the chatbot as "a very early beta product – the best we could do with 2 months of training" that could "improve rapidly with each passing week".[13]

On March 11, 2024, Musk posted on X that the language model would go open source within a week. Six days later, on March 17, Grok-1 was open sourced under the Apache-2.0 license.[14][15] Disclosed were the networks architecture and its weight parameters.[16]

On March 26, 2024, Musk announced that Grok would be enabled for premium subscribers, not just those on the higher-end tier, Premium+.[17]

Grok-1.5
Grok-1.5
Developer(s)	xAI
Initial release	May 15, 2024; 10 months ago
Predecessor	Grok-1.5
Successor	Grok-2
Type	
Large language model
Foundation model
License	Proprietary
Website	x.ai/blog/grok-1.5
On March 29, 2024, Grok-1.5 was announced, with "improved reasoning capabilities" and a context length of 128,000 tokens.[18] Grok-1.5 was released to all X Premium users on May 15, 2024.[1]

On April 4, 2024, an update to X's "Explore" page included summaries of breaking news stories written by Grok, a task previously assigned to a human curation team.[19]

On April 12, 2024, Grok-1.5 Vision (Grok-1.5V) was announced. Grok-1.5V is able to process a wide variety of visual information, including documents, diagrams, graphs, screenshots, and photographs.[20] Grok-1.5V was never released to the public.

On May 4, 2024, Grok became available in the United Kingdom,[21] that being the only country in Europe to support Grok at the moment due to the impending Artificial Intelligence Act rules in the European Union. Grok was later reviewed by the EU and was released on May 16, 2024.[22]"""

In [18]:
prompt1=PromptTemplate(
    template="analysis the the given text carefully {text} and take the necessary data",
    input_variables=["text"]
)

prompt2=PromptTemplate(
    template="summarize the given text in 2 bullet points {text}",
    input_variables=["text"]
)

In [19]:
chain = prompt1 | gemini_model | parser | prompt2 | gemini_model | parser

In [20]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

In [21]:
result=chain.invoke({"text":text})
print(result)

Here are two bullet points summarizing the provided text about Grok:

*   Grok underwent rapid development from an early beta in November 2023 to more advanced versions like Grok-1.5 by mid-2024, incorporating improved reasoning, expanded context, and features such as visual processing and news summarization on X.
*   Its access strategy evolved from a restricted preview to all X Premium users, with its foundational Grok-1 model being open-sourced, and its availability strategically expanding to regions like the UK and EU.


## **Parallel Chain**

In [22]:
prompt1=PromptTemplate(
    template="generate simple summary from the following text \n {text}",
    input_variables=["text"]
)

prompt2=PromptTemplate(
    template="generate 3 question and answer from the following text \n {text}",
    input_variables=["text"]
    )

prompt3=PromptTemplate(
    template="analysis the summary and qa and generate the 5 important quiz with 4 possible answer \n summary: {summary}, Q&A: {qa}",
    input_variables=["summary","qa"]
)

In [24]:
from langchain.schema.runnable import RunnableParallel

parallel_chain=RunnableParallel({
    "summary": prompt1 | gemini_model | parser,
    "qa" : prompt2 | gemini_model | parser
})

parallel_chain

{
  summary: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='generate simple summary from the following text \n {text}')
           | ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-preview-05-20', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B27CAC43E0>, default_metadata=(), model_kwargs={})
           | StrOutputParser(),
  qa: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='generate 3 question and answer from the following text \n {text}')
      | ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-preview-05-20', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B27CAC43E0>, default_metadata=(), model_kwargs={})
      | StrOutputParser()
}

In [25]:
parallel_chain.invoke({"text":text})

{'summary': "Grok, xAI's chatbot, began previewing in November 2023 to selected X Premium users. Initially intended for higher-tier subscribers, its availability expanded to all X Premium users by March 2024, when Grok-1 was also made open-source.\n\nThe improved Grok-1.5 was announced in March 2024 and released in May 2024. Grok also started summarizing news on X in April 2024. While a visual version (Grok-1.5V) was announced, it was never publicly released. Grok's availability expanded to the UK and later the EU in May 2024.",
 'qa': 'Here are 3 questions and answers from the provided text:\n\n1.  **Question:** When was Grok-1 open-sourced and under what license?\n    **Answer:** Grok-1 was open-sourced on March 17, 2024, under the Apache-2.0 license.\n\n2.  **Question:** What new capabilities were announced with Grok-1.5?\n    **Answer:** Grok-1.5 was announced with "improved reasoning capabilities" and a context length of 128,000 tokens.\n\n3.  **Question:** What new role did Grok 

```
{'summary': "Grok, xAI's chatbot, began previewing in November 2023 to selected X Premium users. Initially intended for higher-tier subscribers, its availability expanded to all X Premium users by March 2024, when Grok-1 was also made open-source.\n\nThe improved Grok-1.5 was announced in March 2024 and released in May 2024. Grok also started summarizing news on X in April 2024. While a visual version (Grok-1.5V) was announced, it was never publicly released. Grok's availability expanded to the UK and later the EU in May 2024.",

'qa': 'Here are 3 questions and answers from the provided text:\n\n1.  **Question:** When was Grok-1 open-sourced and under what license?\n    **Answer:** Grok-1 was open-sourced on March 17, 2024, under the Apache-2.0 license.\n\n2.  **Question:** What new capabilities were announced with Grok-1.5?\n    **Answer:** Grok-1.5 was announced with "improved reasoning capabilities" and a context length of 128,000 tokens.\n\n3.  **Question:** What new role did Grok take on for X\'s "Explore" page starting April 4, 2024?\n    **Answer:** On April 4, 2024, Grok began writing summaries of breaking news stories for X\'s "Explore" page, a task previously assigned to a human curation team.'}
```

In [26]:
chain= parallel_chain | prompt3 | gemini_model | parser

In [27]:
chain.get_graph().print_ascii()

                    +---------------------------+                      
                    | Parallel<summary,qa>Input |                      
                    +---------------------------+                      
                       ***                   ***                       
                   ****                         ****                   
                 **                                 **                 
    +----------------+                          +----------------+     
    | PromptTemplate |                          | PromptTemplate |     
    +----------------+                          +----------------+     
             *                                           *             
             *                                           *             
             *                                           *             
+------------------------+                  +------------------------+ 
| ChatGoogleGenerativeAI |                  | ChatGoogleGenerati

In [28]:
text="""AI agent or simply agent), expands this concept by proactively pursuing goals, making decisions, and taking actions over extended periods, thereby exemplifying a novel form of digital agency.[1]

Intelligent agents can range from simple to highly complex. A basic thermostat or control system is considered an intelligent agent, as is a human being, or any other system that meets the same criteria—such as a firm, a state, or a biome.[2]

Intelligent agents operate based on an objective function, which encapsulates their goals. They are designed to create and execute plans that maximize the expected value of this function upon completion.[3] For example, a reinforcement learning agent has a reward function, which allows programmers to shape its desired behavior.[4] Similarly, an evolutionary algorithm's behavior is guided by a fitness function.[5]

Intelligent agents in artificial intelligence are closely related to agents in economics, and versions of the intelligent agent paradigm are studied in cognitive science, ethics, and the philosophy of practical reason, as well as in many interdisciplinary socio-cognitive modeling and computer social simulations.

Intelligent agents are often described schematically as abstract functional systems similar to computer programs. To distinguish theoretical models from real-world implementations, abstract descriptions of intelligent agents are called abstract intelligent agents. Intelligent agents are also closely related to software agents—autonomous computer programs that carry out tasks on behalf of users. They are also referred to using a term borrowed from economics: a "rational agent".[2]"""

In [29]:
result=chain.invoke({"text": text})

print(result)

Here are 5 important quiz questions with 4 possible answers each, based on the provided summary and Q&A:

---

**Quiz on AI Agents**

1.  **What is the primary characteristic that defines an AI agent?**
    a) Its ability to process large amounts of data quickly.
    b) Its capacity to proactively pursue goals, make decisions, and take actions over time.
    c) Its reliance on human input for every action.
    d) Its exclusive application in complex, human-like scenarios.

2.  **How do intelligent agents primarily operate to achieve their goals?**
    a) By reacting randomly to external stimuli until a goal is met.
    b) By being directly controlled by a human operator for every step.
    c) By creating and executing plans designed to maximize the expected value of an objective function.
    d) By passively waiting for instructions before taking any action.

3.  **Which of the following best represents the range of complexity for AI agents, from simple to complex?**
    a) A calculato